Exploratory Data Analysis 

Below are all the features of the dataset before I do anything.
(***Measurement are in meters***)

- amount_tsh - Total static head (amount water available to waterpoint)
- date_recorded - The date the row was entered
- funder - Who funded the well
- gps_height - Altitude of the well
- installer - Organization that installed the well
- longitude - GPS coordinate
- latitude - GPS coordinate
- wpt_name - Name of the waterpoint if there is one
- num_private -
- basin - Geographic water basin
- subvillage - Geographic location
- region - Geographic location
- region_code - Geographic location (coded)
- district_code - Geographic location (coded)
- lga - Geographic location
- ward - Geographic location
- population - Population around the well
- public_meeting - True/False
- recorded_by - Group entering this row of data
- scheme_management - Who operates the waterpoint
- scheme_name - Who operates the waterpoint
- permit - If the waterpoint is permitted
- construction_year - Year the waterpoint was constructed
- extraction_type - The kind of extraction the waterpoint uses
- extraction_type_group - The kind of extraction the waterpoint uses
- extraction_type_class - The kind of extraction the waterpoint uses
- management - How the waterpoint is managed
- management_group - How the waterpoint is managed
- payment - What the water costs
- payment_type - What the water costs
- water_quality - The quality of the water
- quality_group - The quality of the water
- quantity - The quantity of water
- quantity_group - The quantity of water
- source - The source of the water
- source_type - The source of the water
- source_class - The source of the water
- waterpoint_type - The kind of waterpoint
- waterpoint_type_group - The kind of waterpoint

In [1]:
import numpy as np
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


Adding in all of my tests and training dataset. The train-test split was already done for us prior to bringing in our data.

In [2]:
#dependent variable lables (y_train labels)
training_set_labels = pd.read_csv('./data/training_set_labels.csv')

I am dropping the ID column in our y_train dataset

In [3]:
training_set_labels = training_set_labels['status_group']

In [4]:
#The independent variables for the training set (X_train)
training_set_values = pd.read_csv('./data/training_set_values.csv')

In [5]:
df = pd.concat([training_set_values, training_set_labels], axis=1, join="outer")

In [6]:
df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,soft,good,enough,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


In [7]:
df['amount_tsh'].describe()

count     59400.000000
mean        317.650385
std        2997.574558
min           0.000000
25%           0.000000
50%           0.000000
75%          20.000000
max      350000.000000
Name: amount_tsh, dtype: float64

There's an outlier in the 'amount_tsh' with our max value being 350,000. That's taller than our stratosphere. There is something wrong with this data.

In [8]:
df.amount_tsh.sort_values(ascending=False).head(7000)

10812    350000.0
22191    250000.0
22817    200000.0
45067    170000.0
9961     138000.0
           ...   
5895        500.0
3806        500.0
47160       500.0
51059       500.0
53984       500.0
Name: amount_tsh, Length: 7000, dtype: float64

In [9]:
df.amount_tsh.values[10812]

350000.0

In [10]:
block_outliers = df['amount_tsh'] < 4500

In [11]:
cleaned_df = df[block_outliers]

In [12]:
cleaned_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
5,9944,20.0,2011-03-13,Mkinga Distric Coun,0,DWE,39.172796,-4.765587,Tajiri,0,...,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59394,11164,500.0,2011-03-09,World Bank,351,ML appro,37.634053,-6.124830,Chimeredya,0,...,soft,good,enough,enough,machine dbh,borehole,groundwater,communal standpipe,communal standpipe,non functional
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,fluoride,fluoride,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,soft,good,insufficient,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


Going through our data, I noticed that quantity and quantity group looked really similiar so I went out and checked the columns.

In [13]:
cleaned_df['quantity'].value_counts()

enough          32491
insufficient    14932
dry              6208
seasonal         4035
unknown           789
Name: quantity, dtype: int64

In [14]:
cleaned_df['quantity_group'].value_counts()

enough          32491
insufficient    14932
dry              6208
seasonal         4035
unknown           789
Name: quantity_group, dtype: int64

My guess was correct. 'quantity' and 'quantity group' were exactly the same! So I decided to drop the 'quantity_group'.

In [15]:
cleaned_df = df.drop('quantity_group', axis=1)

In [16]:
cleaned_df

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,...,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,69572,6000.0,2011-03-14,Roman,1390,Roman,34.938093,-9.856322,none,0,...,annually,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,8776,0.0,2013-03-06,Grumeti,1399,GRUMETI,34.698766,-2.147466,Zahanati,0,...,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,34310,25.0,2013-02-25,Lottery Club,686,World vision,37.460664,-3.821329,Kwa Mahundi,0,...,per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,67743,0.0,2013-01-28,Unicef,263,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,0,...,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,19728,0.0,2011-07-13,Action In A,0,Artisan,31.130847,-1.825359,Shuleni,0,...,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,60739,10.0,2013-05-03,Germany Republi,1210,CES,37.169807,-3.253847,Area Three Namba 27,0,...,per bucket,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,27263,4700.0,2011-05-07,Cefa-njombe,1212,Cefa,35.249991,-9.070629,Kwa Yahona Kuvala,0,...,annually,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,37057,0.0,2011-04-11,NaN,0,NaN,34.017087,-8.750434,Mashine,0,...,monthly,fluoride,fluoride,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,31282,0.0,2011-03-08,Malec,0,Musa,35.861315,-6.378573,Mshoro,0,...,never pay,soft,good,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


Starting to drop off the columns. Now since I dropped my duplicates, I need to start dropping off the columns I do not need. I chose these datasets after reading the description and made the judgement that these values will not interfere with predicting my target value. Also, I made a judgement call

In [17]:
cleaned_df.drop(['date_recorded','latitude', 'longitude', 'id','recorded_by', 'num_private', 'scheme_name', 'subvillage'], axis = 1, inplace=True)

After dropping Columns, I am checking to see how many null values are in my dataset.

In [18]:
cleaned_df.isnull().sum().sum()

17557

In [19]:
cleaned_df

,amount_tsh,funder,gps_height,installer,wpt_name,basin,region,region_code,district_code,lga,...,payment_type,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group
0,6000.0,Roman,1390,Roman,none,Lake Nyasa,Iringa,11,5,Ludewa,...,annually,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
1,0.0,Grumeti,1399,GRUMETI,Zahanati,Lake Victoria,Mara,20,2,Serengeti,...,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
2,25.0,Lottery Club,686,World vision,Kwa Mahundi,Pangani,Manyara,21,4,Simanjiro,...,per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional
3,0.0,Unicef,263,UNICEF,Zahanati Ya Nanyumbu,Ruvuma / Southern Coast,Mtwara,90,63,Nanyumbu,...,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional
4,0.0,Action In A,0,Artisan,Shuleni,Lake Victoria,Kagera,18,1,Karagwe,...,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59395,10.0,Germany Republi,1210,CES,Area Three Namba 27,Pangani,Kilimanjaro,3,5,Hai,...,per bucket,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional
59396,4700.0,Cefa-njombe,1212,Cefa,Kwa Yahona Kuvala,Rufiji,Iringa,11,4,Njombe,...,annually,soft,good,enough,river,river/lake,surface,communal standpipe,communal standpipe,functional
59397,0.0,NaN,0,NaN,Mashine,Rufiji,Mbeya,12,7,Mbarali,...,monthly,fluoride,fluoride,enough,machine dbh,borehole,groundwater,hand pump,hand pump,functional
59398,0.0,Malec,0,Musa,Mshoro,Rufiji,Dodoma,1,4,Chamwino,...,never pay,soft,good,insufficient,shallow well,shallow well,groundwater,hand pump,hand pump,functional


Now I want to go over each feature and see how many nulls are in each column.

In [20]:
cleaned_df.isna().sum()

amount_tsh                  0
funder                   3635
gps_height                  0
installer                3655
wpt_name                    0
basin                       0
region                      0
region_code                 0
district_code               0
lga                         0
ward                        0
population                  0
public_meeting           3334
scheme_management        3877
permit                   3056
construction_year           0
extraction_type             0
extraction_type_group       0
extraction_type_class       0
management                  0
management_group            0
payment                     0
payment_type                0
water_quality               0
quality_group               0
quantity                    0
source                      0
source_type                 0
source_class                0
waterpoint_type             0
waterpoint_type_group       0
status_group                0
dtype: int64

In [21]:
cleaned_df['funder'].value_counts()

Government Of Tanzania        9084
Danida                        3114
Hesawa                        2202
Rwssp                         1374
World Bank                    1349
                              ... 
Mhuzu                            1
Kanamama                         1
Comunity Construction Fund       1
Mbuzi Mawe                       1
Mzee Mkungata                    1
Name: funder, Length: 1897, dtype: int64

In [22]:
cleaned_df['installer'].value_counts()

DWE                   17402
Government             1825
RWE                    1206
Commu                  1060
DANIDA                 1050
                      ...  
ABDALA                    1
KILANGANI CO              1
maendeleo ya jamii        1
Raurensia                 1
Word                      1
Name: installer, Length: 2145, dtype: int64

In [23]:
cleaned_df['public_meeting'].value_counts

<bound method IndexOpsMixin.value_counts of 0        True
1         NaN
2        True
3        True
4        True
         ... 
59395    True
59396    True
59397    True
59398    True
59399    True
Name: public_meeting, Length: 59400, dtype: object>

In [24]:
cleaned_df['permit'].value_counts

<bound method IndexOpsMixin.value_counts of 0        False
1         True
2         True
3         True
4         True
         ...  
59395     True
59396     True
59397    False
59398     True
59399     True
Name: permit, Length: 59400, dtype: object>

We handled our outliers and we have 5 features that have null values. We will handle the null values later.

We are starting our Train-Test Split

In [25]:
X = cleaned_df.drop('status_group', axis=1)
y = cleaned_df['status_group']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

We are building our dummy model.

In [27]:
dummy_model = DummyClassifier(strategy= 'most_frequent')
dummy_model.fit(X_train, y_train)
dummy_model.predict(X_train)[:50]

array(['functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional', 'functional', 'functional',
       'functional', 'functional'], dtype='<U10')

In [28]:
dummy_model.score(X_test, y_test)

0.5405387205387205

We got our dummy model. We are now going to start using pipelines.

In [29]:
#We have to start making our simple functions to start on our pipelines.
def grab_numeric(df):
    return df.select_dtypes(include=['float', 'int'])


def grab_object(df): 
    return df.select_dtypes(include=['object'])

In [30]:
#We are now making a function transformer so that we can fit this into our pipeline.


#Function Transformer for Numeric Values
GrabNumeric = FunctionTransformer(grab_numeric)


#Function Transformer for Object Values
GrabObject = FunctionTransformer(grab_object)

In [31]:
#We are building our pipeline here.
pipe = Pipeline(steps=[
    ('num', GrabNumeric), #for a pipeline to work, you must have a fit and a transform method.
    ('ss', StandardScaler() )
    #dont need to fit and transform our standard scaler
    
])

In [32]:
#We are now fitting and transforming our pipeline on our X_train.
pipe.fit_transform(X_train)

array([[-0.12679411,  0.99879067,  0.09442061, -0.48002817,  0.25054689,
         0.71544987],
       [-0.12679411, -0.96894381,  0.09442061, -0.06629284, -0.37677904,
        -1.37368892],
       [-0.12679411, -0.96894381,  0.15110581, -0.37659434, -0.37677904,
        -1.37368892],
       ...,
       [ 0.07543366,  1.16036418,  0.03773542, -0.37659434,  0.11462627,
         0.73965651],
       [ 0.47988919,  0.08705446, -0.30237575, -0.37659434, -0.37677904,
         0.73334174],
       [-0.10657133, -0.9891405 ,  4.7426066 , -0.48002817,  0.56420986,
         0.73755159]])

In [33]:
number_mini_pipeline = Pipeline(steps=[('get_num', GrabNumeric),
                              ('num_impute', SimpleImputer()),
                           ('ss', StandardScaler())])
category_mini_pipeline = Pipeline(steps=[('cat_impute', SimpleImputer(strategy='most_frequent')),
                             ('ohe', OneHotEncoder(sparse=False, handle_unknown='ignore'))])

In [34]:
#This is a column transformer using pipelines
#I need to fix these values.
ct = ColumnTransformer(transformers = [
    ('subpipe_num', subpipe_num, X_train.columns),
    ('subpipe_cat', subpipe_cat, [0,5,7,8]),
    ('subpipe_ord', subpipe_ord, [1])
])

NameError: name 'subpipe_num' is not defined